<a href="https://colab.research.google.com/github/sam80FDS/Sam/blob/main/Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
#  HEADLESS STUDIO v27.0 (The Nuclear Fix)
#  Engine: ComfyUI + GitMyLo (Audio) + ParlerTTS
#  Strategy: Deletes conflicting manifest files to force installation.
# ==============================================================================

import os
import time

print("------------------------------------------------")
print("STEP 1: SYSTEM LAYER")
print("------------------------------------------------")
!apt-get update
!apt-get install -y ffmpeg libsndfile1 libavcodec-dev libavformat-dev libavutil-dev libswscale-dev libavdevice-dev pkg-config

print("------------------------------------------------")
print("STEP 2: APPLICATION LAYER")
print("------------------------------------------------")
%cd /content
!rm -rf ComfyUI
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip install -r requirements.txt

print("------------------------------------------------")
print("STEP 3: NODE INJECTION")
print("------------------------------------------------")
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git
!git clone https://github.com/gitmylo/ComfyUI-audio-nodes.git
!git clone https://github.com/smthemex/ComfyUI_ParlerTTS.git

print("------------------------------------------------")
print("STEP 4: FORCED DEPENDENCY INSTALLATION (The Fix)")
print("------------------------------------------------")
# 1. Enter Audio Node Directory
%cd /content/ComfyUI/custom_nodes/ComfyUI-audio-nodes

# 2. NUCLEAR STRIKE: Delete the file causing the "torch==2.1.0" error.
!rm requirements.txt

# 3. MANUAL INSTALLATION (Safe List)
# We install only the safe packages, skipping the bad torch requirements.
!pip install soundfile numpy scipy librosa

# 4. FORCE AUDIOCRAFT (No Dependencies Mode)
# This installs the engine without checking driver versions.
!pip install --no-deps git+https://github.com/facebookresearch/audiocraft.git

# 5. Install ParlerTTS Deps
%cd ../ComfyUI_ParlerTTS
!pip install -r requirements.txt
%cd ..

print("------------------------------------------------")
print("STEP 5: ASSET DOWNLOAD")
print("------------------------------------------------")
%cd /content/ComfyUI/models/checkpoints
!wget -nc -O dreamshaper_8.safetensors https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors
!wget -nc -O svd_xt.safetensors https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt.safetensors
!wget -nc -O musicgen_small.safetensors https://huggingface.co/facebook/musicgen-small/resolve/main/model.safetensors

!mkdir -p /content/ComfyUI/models/parler_tts
!wget -nc -O /content/ComfyUI/models/parler_tts/parler-tts-mini-v1.safetensors https://huggingface.co/parler-tts/parler-tts-mini-v1/resolve/main/model.safetensors

!mkdir -p /content/ComfyUI/models/upscale_models
!wget -nc -O /content/ComfyUI/models/upscale_models/RealESRGAN_x4plus.pth https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
!mkdir -p /content/ComfyUI/models/vfi
!wget -nc -O /content/ComfyUI/models/vfi/rife47.pth https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rife47.pth

print("------------------------------------------------")
print("STEP 6: LAUNCH")
print("------------------------------------------------")
%cd /content/ComfyUI
!wget -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

print("ENGINE INITIALIZING... WAIT FOR THE LINK")
!python main.py --listen --dont-print-server & cloudflared tunnel --url http://localhost:8188


------------------------------------------------
STEP 1: SYSTEM LAYER
------------------------------------------------
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://cli.github.com/packages stable InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:12 http://archive.ubuntu